In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import io
import os
import sklearn # 머신러닝
from sklearn.metrics.pairwise import cosine_similarity
# 맥os의 경우 시도때도없이 다운되서 아래와 같이 넣어준다.
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
#파일업로드
myFirstDf = files.upload()
mySecondDf = files.upload()

Saving first.csv to first.csv


Saving third.csv to third.csv


In [3]:
#데이터프레임으로 변경
recipeData = pd.read_csv(io.BytesIO(myFirstDf['first.csv']))
userData = pd.read_csv(io.BytesIO(mySecondDf['third.csv']))

In [4]:
#필요없는 column 제거
recipeData = recipeData.drop(columns=['Unnamed: 0','serving','coockingTime','difficult','recipe_source','menu_img','recipe_category'],axis=1)

In [5]:
#필요없는 column 제거
userData = userData.drop(columns= ['Unnamed: 0','created_day'])

In [6]:
#결측값 제거
userData = userData.dropna(axis = 0)

In [7]:
#이름 순으로 정렬
userData = userData.sort_values(by='nick',ascending=True)

In [8]:
#userId 추가
userData['userId'] = ""

In [9]:
#rId와 grade 값을 정수형으로 변환
userData[['rId','grade']] = userData[['rId','grade']].astype(int)

In [10]:
#userData에서 nick이 공란일 경우 모두 제거
userData = userData[userData.nick != ' ']

In [ ]:
#공백제거
userData['nick'].str.strip()

In [12]:
#userId를 넣기 위한 인덱스 초기화
userData = userData.reset_index()
userData = userData.drop(columns=['index'],axis = 1)

In [ ]:
#User ID 부여 - 중복방지
userID = 2

for i in range(277441):
  if i % 10000 == 0 :
    print(i)
  if i == 0 or i == 1 :
    userData['userId'][i] = userID - 1
  elif userData['nick'][i] != userData['nick'][i+1] :
    userData['userId'][i] = userID
    userID = userID + 1
  else :
    userData['userId'][i] = userID

In [ ]:
#누락 추가
userData['userId'][277441] = 93370

In [15]:
#csv파일로 저장시 '-'로인한 오류발생 -> 제거
userData['nick'] = userData['nick'].str.replace("-","")

In [ ]:
#rId를 기준으로 정렬
recipeData.sort_values(by='rId',ascending=True)

In [17]:
#CSV파일로 저장!
UserRating = userData.to_csv("UserRating.csv",mode = 'w',encoding='utf-8-sig')

In [18]:
#CSV파일로 저장!
recipeDataSet = recipeData.to_csv("recipeDataSet.csv",mode = 'w',encoding='utf-8-sig')

In [2]:
#업로드
myFirstDf = files.upload()
mySecondDf = files.upload()

#데이터프레임으로 변경
recipeData = pd.read_csv(io.BytesIO(myFirstDf['recipeDataSet.csv']))
userData = pd.read_csv(io.BytesIO(mySecondDf['UserRating.csv']))

Saving recipeDataSet.csv to recipeDataSet.csv


Saving UserRating.csv to UserRating.csv


In [3]:
# 두 데이터프레임을 합쳐보자
userRecipeRating = pd.merge(userData,recipeData,on = 'rId')

In [4]:
# 닉네임은 필요 없으니 여기서는 제거해주자
userRecipeRating = userRecipeRating.drop('nick',axis = 1)
# userRecipeRating = userRecipeRating.drop('recipe_title',axis = 1)
userRecipeRating = userRecipeRating.drop('Unnamed: 0_x',axis = 1)
userRecipeRating = userRecipeRating.drop('Unnamed: 0_y',axis = 1)

In [5]:
#개수 줄여야 돌아간다..
userRecipeRating = userRecipeRating[:190000]

In [6]:
userRecipeRating['recipe_title'].str.strip()

0         얼갈이배추로 아삭아삭! 시원~한 얼갈이겉절이 만들어요.(배추겉절이)
1         얼갈이배추로 아삭아삭! 시원~한 얼갈이겉절이 만들어요.(배추겉절이)
2         얼갈이배추로 아삭아삭! 시원~한 얼갈이겉절이 만들어요.(배추겉절이)
3         얼갈이배추로 아삭아삭! 시원~한 얼갈이겉절이 만들어요.(배추겉절이)
4         얼갈이배추로 아삭아삭! 시원~한 얼갈이겉절이 만들어요.(배추겉절이)
                          ...                  
189995                          깻잎김치와는 다른맛~ 깻잎찜
189996                          깻잎김치와는 다른맛~ 깻잎찜
189997                          깻잎김치와는 다른맛~ 깻잎찜
189998                          깻잎김치와는 다른맛~ 깻잎찜
189999                          깻잎김치와는 다른맛~ 깻잎찜
Name: recipe_title, Length: 190000, dtype: object

In [7]:
# 아이템 기반 협업 필터링을 위해서는 pivot table을 만들어야 한다.
# 즉, 사용자 - 레시피에 따른 평점 점수가 데이터로 들어가게 된다.
# 데이터 분포는 다음과 같다.
# data : 레시피 평점 rating
# index : 레시피 title
# columns : userId

# 마찬가지로 Over Flow 발생
# recipeUserRating = userRecipeRating.groupby(['userId','recipe_title'])['grade'].max().unstack()
# recipeUserRating = userRecipeRating.pivot_table('grade',index='rId',columns='userId')

# Over Flow 발생 안함 -> 개수를 줄여야가능
recipeUserRating = userRecipeRating.pivot_table('grade',index = 'recipe_title', columns='userId')

In [8]:
#N/A 값들을 0으로 대체
recipeUserRating.fillna(0,inplace=True)

In [9]:
#코사인유도
itemBasedCollabor = cosine_similarity(recipeUserRating)
itemBasedCollabor

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
#인덱스/컬럼 itemBasedCollabor 값으로 넣기
itemBasedCollabor = pd.DataFrame(data = itemBasedCollabor,index = recipeUserRating.index, columns= recipeUserRating.index)

In [23]:
#키로 검색시 에러방지를 위해 좌우 스트립
itemBasedCollabor.columns = itemBasedCollabor.columns.str.strip()
itemBasedCollabor.index =  itemBasedCollabor.index.str.strip()

In [25]:
#CSV로 저장
# itemBasedCollabor.to_csv("collabor.csv",mode = 'w',encoding='utf-8-sig')

In [26]:
#특정 아이템을 선호한다면 함께 보여줄 아이템 리스트 6개정도를 보여주는 함수
def getItemBasedCollabor(recipe_title):
  return itemBasedCollabor[recipe_title].sort_values(ascending = False)[:20]

In [27]:
#예를 들어 10분 밑반찬/굴소스 애호박볶음을 선호한다면 다음 6개를 관련된 리스트로 보여줄 수 있음. 다만, 정확도가 아직 떨어져보인다.
getItemBasedCollabor('10분 밑반찬/굴소스 애호박볶음')

recipe_title
10분 밑반찬/굴소스 애호박볶음                            1.000000
[백종원레시피]백종원 제육볶음                             0.250000
전복미역국 끓이기                                    0.217597
목살을 넣어 만든 돼지고기 김치찜                           0.204124
쏘세지 야채볶음 맛있을수밖에~                             0.144338
가지호박볶음(덮밥):고기한 점 없어서 더 좋은 여름채소볶음             0.140193
쪽파오이무침:아삭아삭 상큼하니 봄김치가 따로없구나(feat.슬라이스아몬드)    0.133631
백종원 챱스테이크(누구나 쉽게 만드는 간단요리 찹스테이크...)          0.125000
오징어젓갈볶음밥 ☆ 초간단, 초스피드                         0.125000
오징어무국 얼큰시원 맛있게 끓이는법~                         0.120281
가을 무로 맛있는 깍두기 담그기                            0.102062
초간단 베이컨&팽이버섯전~*                              0.094491
참치김치찌개 맛있게 끓이는법 이거 넣기~!                      0.081890
고기없이 비지찌개 끓이기                                0.081111
알리오올리오 베이컨 파스타                               0.075723
오이지 무침 꼬들꼬들 아삭하게 만드는법                        0.073850
백종원 비빔국수로 혼밥~                                0.072411
바삭바삭한 꽈리고추멸치볶음                               0.069231
데리야끼 닭봉조림      